In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-12-27 15:10:43--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2024-12-27 15:10:43 (14.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
!pip install -r requirements.txt

In [8]:
with open('input.txt', 'r') as file:
    text = file.read()

print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [9]:
print(len(text))

1115394


In [13]:
chars = sorted(list(set(text)))
print(''.join(chars))
vocab_size = len(chars)
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [53]:
##character level language model

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

def encode(s):
    return [stoi[c] for c in s]

def decode(l):
    if isinstance(l, torch.Tensor):
        l = l.tolist()
    if isinstance(l, int):
        return itos[l]
    return ''.join([itos[i] for i in l])

print(encode('hii my name is yash'))
print(decode(encode('hii my name is yash')))



[46, 47, 47, 1, 51, 63, 1, 52, 39, 51, 43, 1, 47, 57, 1, 63, 39, 57, 46]
hii my name is yash


In [54]:
# import tiktoken

# enc = tiktoken.get_encoding('gpt2')

# print(enc.encode('yash'))
# print(enc.decode(enc.encode('yash')))

In [55]:
#encode the shakespeare text

import torch

data = torch.tensor(encode(text), dtype=torch.long) #torch.long is equivalent to torch.int64
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [56]:
#train-val split 90-10
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


In [57]:
block_size = 8
decode(train_data[block_size:block_size+1])

't'

In [58]:
x = train_data[:block_size+1]
y = train_data[1:block_size+2]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {decode(context)} the target is {decode(target)}")

when input is F the target is i
when input is Fi the target is r
when input is Fir the target is s
when input is Firs the target is t
when input is First the target is  
when input is First  the target is C
when input is First C the target is i
when input is First Ci the target is t


In [62]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    random_indices = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in random_indices])
    y = torch.stack([data[i+1:i+block_size+1] for i in random_indices])
    return x, y

x, y = get_batch('train')
print(x.shape, y.shape)

torch.Size([4, 8]) torch.Size([4, 8])


In [138]:
import torch.nn as nn
import torch.nn.functional as F

n_embed = 32
num_heads = 4

class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        #compute attention scores
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        #aggregation of values
        out = wei @ v
        return out
    
class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1) #concatenating over the channel dimension
    
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.selfattention_heads = MultiHeadAttention(num_heads, n_embed//num_heads) #i.e. 4 heads of 8-dimensional self-attention
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        #idx and targets are both (B, T) tensor of integers
        B, T = idx.shape
        token_embeddings = self.token_embedding_table(idx) #B, T, n_embed(C)
        position_embeddings = self.position_embedding_table(torch.arange(T)) #T, n_embed
        x = token_embeddings + position_embeddings #B, T, n_embed
        x = self.selfattention_heads(x) #apply one head of self-attention
        logits = self.lm_head(x) #B, T, vocab_size

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            #crop idx to last block_size tokens
            idx_cond = idx[:, -block_size:]
            #get predictions
            logits, loss = self(idx_cond)
            #focus only on last time step
            logits = logits[:, -1, :]
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            #sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=-1)
        return idx
    
model = BigramLanguageModel()

eval_iters = 20
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# logits, loss = model(x, y)

# print(logits.shape)
# print(loss)

# idx = torch.zeros((1, 1), dtype=torch.long) #starting with a newline token
# print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))




In [139]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [140]:
batch_size = 32
for steps in range(2000):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(f"loss {loss.item()}")


loss 2.3119101524353027


In [141]:
idx = torch.zeros((1, 1), dtype=torch.long) #starting with a newline token
print(decode(model.generate(idx, max_new_tokens=1000)[0].tolist()))


D pt RBol nas dorno mor bin oth me histwece, lhantericat ve the!r Afnd achn; icooren the wig yhigh morga furus wourrtr ne.

Nhix per:
ow athl he,
Whigan!
Sort eiisenta,
Ther.

ouch duy they meosdit,
Andis tilthe loud fovend youth not'o I,
CI I I fore ste than; Esd, wrre!
I hin se dhasrsige!
NTARENID:
Wha forenl to orenth equae brrelll hrofin out hakesn,
Feachk tond ladts lor
I ne chouly; hiilecr'che priese son che pre ud..

WAUDI I de wre thwons voll be, all yi? weacile.

Gol:
Anxe ol owefevere trecn wkey bete ass yimbornt swaxd and the ghingch and wort hant a sillhiv tho soth coith nge hay.

UWADor cist seter: no is cont ter. ancre,
Lor a grof to fuyay hancar, row berl ara wore bont-th sffrevolered anriset.


NOORA
TARNDL:
ETAUS:
TUIEXe yot rorvied,
Wlor: Sies; ben.


D!
LIUSoll, homp ou jat che cere thad best ere s dhe licd athe!
LINf:
Har meaner berrand he;
Asmade,

DUQYATWRYwes tel pios keencomepse,

Fuolasnr waed gh hum spuaam to fait noto ourid'e'clou sy twator'th ve me!
O doay 

In [112]:
import numpy as np

-np.log(1/65)
#this is the expected loss for a random guess, 
# but since we are getting 4.27 it means our model has some extra entropy and the loss is higher than expected


np.float64(4.174387269895637)

# Attention

In [97]:
 #toy example

torch.manual_seed(1337)
B,T,C = 4,8,32

x = torch.randn(B,T,C)

# single head self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) #(B,T,16)
q = query(x) #(B, T, 16)
v = value(x) #(B, T, 16)

weights = q @ k.transpose(-2, -1) #(B,T,16) @ (B,16,T) -> (B,T,T)

tril = torch.tril(torch.ones(T,T))
# weights = torch.zeros(T,T)
weights = weights.masked_fill(tril==0, float('-inf'))
weights = F.softmax(weights, dim=-1)
out = weights @ v

out.shape


torch.Size([4, 8, 16])

In [96]:
weights

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089